In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import dgl.function as fn
from functools import partial

In [2]:
from dgl.contrib.data import load_data

In [3]:
data = load_data(dataset='aifb')
num_nodes = data.num_nodes
num_rels = data.num_rels
num_classes = data.num_classes
labels = data.labels
train_idx = data.train_idx
# split training and validation set
val_idx = train_idx[:len(train_idx) // 5]
train_idx = train_idx[len(train_idx) // 5:]

# edge type and normalization factor
edge_type = torch.from_numpy(data.edge_type)
edge_norm = torch.from_numpy(data.edge_norm).unsqueeze(1)

labels = torch.from_numpy(labels).view(-1)

Loading dataset aifb
Number of nodes:  8285
Number of edges:  66371
Number of relations:  91
Number of classes:  4
removing nodes that are more than 3 hops away


In [5]:
dir(data)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'dir',
 'edge_dst',
 'edge_norm',
 'edge_src',
 'edge_type',
 'labels',
 'load',
 'name',
 'num_classes',
 'num_nodes',
 'num_rels',
 'test_idx',
 'train_idx']

In [6]:
data.labels

matrix([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]])

In [7]:
data.labels.shape

(8285, 1)

In [10]:
max(data.labels)

matrix([[3]])

In [14]:
data.edge_dst.shape

(65439,)

In [17]:
data.edge_norm.shape

(65439,)

In [18]:
torch.arange(10)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [20]:
data.edge_dst

array([   0,    0,    1, ..., 8283, 8284, 8284])

In [21]:
data.edge_dst.shape

(65439,)

In [22]:
data.num_rels

91

In [26]:
class RGCNLayer(nn.Module):
    def __init__(self, in_feat, out_feat, num_rels, num_bases=-1, bias=None,
                 activation=None, is_input_layer=False):
        super(RGCNLayer, self).__init__()
        self.in_feat = in_feat
        self.out_feat = out_feat
        self.num_rels = num_rels
        self.num_bases = num_bases
        self.bias = bias
        self.activation = activation
        self.is_input_layer = is_input_layer

        # sanity check
        if self.num_bases <= 0 or self.num_bases > self.num_rels:
            self.num_bases = self.num_rels

        # weight bases in equation (3)
        self.weight = nn.Parameter(torch.Tensor(self.num_bases, self.in_feat,
                                                self.out_feat))
        if self.num_bases < self.num_rels:
            # linear combination coefficients in equation (3)
            self.w_comp = nn.Parameter(torch.Tensor(self.num_rels, self.num_bases))

        # add bias
        if self.bias:
            self.bias = nn.Parameter(torch.Tensor(out_feat))

        # init trainable parameters
        nn.init.xavier_uniform_(self.weight,
                                gain=nn.init.calculate_gain('relu'))
        if self.num_bases < self.num_rels:
            nn.init.xavier_uniform_(self.w_comp,
                                    gain=nn.init.calculate_gain('relu'))
        if self.bias:
            nn.init.xavier_uniform_(self.bias,
                                    gain=nn.init.calculate_gain('relu'))

    def forward(self, g):
        if self.num_bases < self.num_rels:
            # generate all weights from bases (equation (3))
            weight = self.weight.view(self.in_feat, self.num_bases, self.out_feat)
            weight = torch.matmul(self.w_comp, weight).view(self.num_rels,
                                                        self.in_feat, self.out_feat)
        else:
            weight = self.weight

        if self.is_input_layer:
            def message_func(edges):
                # for input layer, matrix multiply can be converted to be
                # an embedding lookup using source node id
                embed = weight.view(-1, self.out_feat)
                index = edges.data['rel_type'] * self.in_feat + edges.src['id']
                return {'msg': embed[index] * edges.data['norm']}
        else:
            def message_func(edges):
                w = weight[edges.data['rel_type']]
                msg = torch.bmm(edges.src['h'].unsqueeze(1), w).squeeze()
                msg = msg * edges.data['norm']
                return {'msg': msg}

        def apply_func(nodes):
            h = nodes.data['h']
            if self.bias:
                h = h + self.bias
            if self.activation:
                h = self.activation(h)
            return {'h': h}

        g.update_all(message_func, fn.sum(msg='msg', out='h'), apply_func)

In [27]:
class Model(nn.Module):
    def __init__(self, num_nodes, h_dim, out_dim, num_rels,
                 num_bases=-1, num_hidden_layers=1):
        super(Model, self).__init__()
        self.num_nodes = num_nodes
        self.h_dim = h_dim
        self.out_dim = out_dim
        self.num_rels = num_rels
        self.num_bases = num_bases
        self.num_hidden_layers = num_hidden_layers

        # create rgcn layers
        self.build_model()

        # create initial features
        self.features = self.create_features()

    def build_model(self):
        self.layers = nn.ModuleList()
        # input to hidden
        i2h = self.build_input_layer()
        self.layers.append(i2h)
        # hidden to hidden
        for _ in range(self.num_hidden_layers):
            h2h = self.build_hidden_layer()
            self.layers.append(h2h)
        # hidden to output
        h2o = self.build_output_layer()
        self.layers.append(h2o)

    # initialize feature for each node
    def create_features(self):
        features = torch.arange(self.num_nodes)
        return features

    def build_input_layer(self):
        return RGCNLayer(self.num_nodes, self.h_dim, self.num_rels, self.num_bases,
                         activation=F.relu, is_input_layer=True)

    def build_hidden_layer(self):
        return RGCNLayer(self.h_dim, self.h_dim, self.num_rels, self.num_bases,
                         activation=F.relu)

    def build_output_layer(self):
        return RGCNLayer(self.h_dim, self.out_dim, self.num_rels, self.num_bases,
                         activation=partial(F.softmax, dim=1))

    def forward(self, g):
        if self.features is not None:
            g.ndata['id'] = self.features
        for layer in self.layers:
            layer(g)
        return g.ndata.pop('h')

In [39]:
# configurations
n_hidden = 16 # number of hidden units
n_bases = -1 # use number of relations as number of bases
n_hidden_layers = 0 # use 1 input layer, 1 output layer, no hidden layer
n_epochs = 25 # epochs to train
lr = 0.01 # learning rate
l2norm = 0 # L2 norm coefficient

# create graph
g = DGLGraph((data.edge_src, data.edge_dst))
g.edata.update({'rel_type': edge_type.long(), 'norm': edge_norm.long()})

# create model
model = Model(len(g),
              n_hidden,
              num_classes,
              num_rels,
              num_bases=n_bases,
              num_hidden_layers=n_hidden_layers)

In [44]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2norm)

print("start training...")
model.train()
for epoch in range(n_epochs):
    optimizer.zero_grad()
    logits = model.forward(g)
    loss = F.cross_entropy(logits[train_idx], labels[train_idx].long())
    loss.backward()

    optimizer.step()

    train_acc = torch.sum(logits[train_idx].argmax(dim=1) == labels[train_idx])
    train_acc = train_acc.item() / len(train_idx)
    val_loss = F.cross_entropy(logits[val_idx], labels[val_idx].long())
    val_acc = torch.sum(logits[val_idx].argmax(dim=1) == labels[val_idx])
    val_acc = val_acc.item() / len(val_idx)
    print("Epoch {:05d} | ".format(epoch) +
          "Train Accuracy: {:.4f} | Train Loss: {:.4f} | ".format(
              train_acc, loss.item()) +
          "Validation Accuracy: {:.4f} | Validation loss: {:.4f}".format(
              val_acc, val_loss.item()))

start training...
Epoch 00000 | Train Accuracy: 0.9554 | Train Loss: 1.3717 | Validation Accuracy: 0.6786 | Validation loss: 1.3810
Epoch 00001 | Train Accuracy: 0.9464 | Train Loss: 1.3520 | Validation Accuracy: 0.7500 | Validation loss: 1.3744
Epoch 00002 | Train Accuracy: 0.9554 | Train Loss: 1.3256 | Validation Accuracy: 0.7500 | Validation loss: 1.3661
Epoch 00003 | Train Accuracy: 0.9554 | Train Loss: 1.2922 | Validation Accuracy: 0.7500 | Validation loss: 1.3549
Epoch 00004 | Train Accuracy: 0.9554 | Train Loss: 1.2534 | Validation Accuracy: 0.7143 | Validation loss: 1.3418
Epoch 00005 | Train Accuracy: 0.9732 | Train Loss: 1.2107 | Validation Accuracy: 0.7500 | Validation loss: 1.3267
Epoch 00006 | Train Accuracy: 0.9732 | Train Loss: 1.1656 | Validation Accuracy: 0.7500 | Validation loss: 1.3099
Epoch 00007 | Train Accuracy: 0.9732 | Train Loss: 1.1198 | Validation Accuracy: 0.7500 | Validation loss: 1.2915
Epoch 00008 | Train Accuracy: 0.9732 | Train Loss: 1.0746 | Validation

In [41]:
logits[train_idx]

tensor([[0.2543, 0.2457, 0.2500, 0.2500],
        [0.2510, 0.2480, 0.2521, 0.2489],
        [0.2520, 0.2508, 0.2501, 0.2472],
        [0.2501, 0.2494, 0.2496, 0.2509],
        [0.2524, 0.2479, 0.2501, 0.2497],
        [0.2503, 0.2497, 0.2519, 0.2481],
        [0.2511, 0.2504, 0.2503, 0.2482],
        [0.2498, 0.2507, 0.2520, 0.2475],
        [0.2531, 0.2478, 0.2491, 0.2500],
        [0.2501, 0.2492, 0.2511, 0.2496],
        [0.2515, 0.2492, 0.2494, 0.2500],
        [0.2514, 0.2494, 0.2494, 0.2498],
        [0.2522, 0.2488, 0.2503, 0.2488],
        [0.2511, 0.2497, 0.2493, 0.2499],
        [0.2499, 0.2506, 0.2493, 0.2502],
        [0.2509, 0.2495, 0.2513, 0.2484],
        [0.2509, 0.2497, 0.2500, 0.2493],
        [0.2512, 0.2494, 0.2497, 0.2497],
        [0.2507, 0.2495, 0.2506, 0.2492],
        [0.2497, 0.2500, 0.2502, 0.2501],
        [0.2492, 0.2501, 0.2520, 0.2487],
        [0.2500, 0.2507, 0.2496, 0.2497],
        [0.2484, 0.2504, 0.2514, 0.2498],
        [0.2478, 0.2510, 0.2509, 0

In [42]:
labels[train_idx]

tensor([2, 1, 2, 2, 1, 2, 1, 0, 3, 3, 1, 0, 2, 2, 1, 3, 2, 2, 1, 2, 0, 2, 0, 2,
        1, 1, 2, 1, 1, 1, 2, 2, 3, 3, 1, 2, 2, 1, 1, 3, 2, 2, 1, 3, 2, 3, 1, 2,
        1, 2, 2, 3, 2, 3, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 0, 2, 2, 2, 3, 1, 3, 3,
        1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 3, 2, 1, 2, 0, 0, 2, 2, 2, 3, 0, 1, 2, 2,
        2, 2, 0, 1, 1, 2, 1, 3, 1, 0, 2, 2, 1, 2, 0, 3], dtype=torch.int32)

In [32]:
dir(g)

['__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_batch_num_edges',
 '_batch_num_nodes',
 '_canonical_etypes',
 '_dsttypes_invmap',
 '_edge_frames',
 '_etype2canonical',
 '_etypes',
 '_etypes_invmap',
 '_find_etypes',
 '_get_e_repr',
 '_get_n_repr',
 '_graph',
 '_idtype_str',
 '_init',
 '_is_unibipartite',
 '_node_frames',
 '_ntypes',
 '_pop_e_repr',
 '_pop_n_repr',
 '_reset_cached_info',
 '_set_e_repr',
 '_set_n_repr',
 '_srctypes_invmap',
 'add_edge',
 'add_edges',
 'add_nodes',
 'add_self_loop',
 'adj',
 'adjacency_matrix',
 'adjacency_matrix_scipy',
 'all_edges',
 'apply_edges',
 'apply_node

In [37]:
x=edge_type.long()

In [46]:
data.edge_src.shape

(65439,)

In [49]:
data.edge_dst.shape

(65439,)

In [54]:
edge_type

tensor([ 0, 37,  0,  ...,  0,  9,  0], dtype=torch.int32)

In [55]:
edge_norm.to

tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])

In [2]:
from dgl.contrib.data import load_data
import torch

In [3]:
data = load_data(dataset='aifb')
num_nodes = data.num_nodes
num_rels = data.num_rels
num_classes = data.num_classes
labels = data.labels
train_idx = data.train_idx
# split training and validation set
val_idx = train_idx[:len(train_idx) // 5]
train_idx = train_idx[len(train_idx) // 5:]

# edge type and normalization factor
edge_type = torch.from_numpy(data.edge_type)
edge_norm = torch.from_numpy(data.edge_norm).unsqueeze(1)

labels = torch.from_numpy(labels).view(-1)

Loading dataset aifb
Number of nodes:  8285
Number of edges:  66371
Number of relations:  91
Number of classes:  4
removing nodes that are more than 3 hops away


In [6]:
train_idx

array([ 505, 6106, 4228, 6641, 1593, 2182, 4844,  780, 4875, 6958, 4080,
       1728, 3564, 5701, 3817, 4760, 4979, 2985,  296, 2771, 5777, 6161,
       6867, 2042,  224, 1220,  660, 2668, 5275, 5915,  483, 7837, 6992,
       4860, 2012, 1457, 1386, 5348, 1647,  792, 5614,  448,  356, 4458,
       6905, 3757, 2456, 5770, 6437, 7165, 7714, 1542, 4330, 3072, 7204,
       6140, 8115, 5577, 6950, 6442, 5273,  693, 5186, 7486, 5367, 2982,
       3474, 4021, 3530,  675, 7990, 5706,  461,  307, 5146, 6355, 1956,
       7117, 2153, 1218, 1594, 7551, 6813, 7447, 3282, 4808, 2630, 3740,
       5360, 3022, 4699,  537, 2325, 3085, 2925,  843, 8217, 4797,  333,
       3235,  440, 5960, 6299, 1012, 8110,  489, 7272, 3470, 7218, 1879,
       7178, 4202])